In [1]:
# Импорт библиотек
import os
from typing import Literal
import json
import cv2
from ultralytics import YOLOWorld
import supervision as sv

## Блок входных данных

In [2]:
# Название используемой модели YOLO-World
model_name = "yolov8x-worldv2.pt"
# Название папки с изображениями
image_dir = "sample_substraction"

# Директория, куда сбрасываются аннотированные изображения (для визуальной проверки результатов)
output_dir = "annotated_images"
# Размер батча
batch_size = 5

In [3]:
# Порог для NMS
nms_threshold = 1e-9
# Порог уверенности распознавания класса
confidence = 0.6

# Размер изображения
image_size = (640, 640)
# Список картинок с относительными путями от запуска скрипта
list_of_images = [os.path.join(image_dir, image) for image in os.listdir(image_dir)]

In [4]:
# Промпт к целевому классу для модели
license_plate_prompt = """
Russian car license plate: 
- white rectangular-shape metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A 123 BC 77 RUS
- mounted with visible bolts/screws
"""
# Промпт к null-классу
null_class_prompt = """"""

# Словарь классов и промптов к ним
classes_dict = {"license_plate": license_plate_prompt,
                "background": null_class_prompt}

# Список лейблов классов
list_of_classes_labels = list(classes_dict.keys())
# Список промптов
list_of_prompts = list(classes_dict.values())

# Название папки с датасетом YOLO
yolo_dataset_name = "test_dataset"

## Функции

In [5]:
def check_output_dir_availability(dir_name: str) -> None:
    """
        Функция для проверки наличия директории, куда сбрасываются размеченные картинки (для их визуальной проверки).

        Args
        ----------
        dir_name (str): название директории для проверки.
    """

    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [6]:
def create_instance_of_detection_dataset(classes_labels: list[str], list_of_images: list[str]) -> sv.DetectionDataset:
    """
        Функция для формирования дата-класса, содержащего информацию о выполненных детекциях 
        для каждого изображения.

        Args
        ----------
        classes_labels (list[str]): список названий классов,
        
        list_of_images (list[str]): список названий файлов изображений 
        (c относительными путями от места запуска скрипта).

        Returns
        -------
        dataset_instance (sv.DetectionDataset): созданнный дата-класс для хранения метаданных о детекциях.
    """
    
    # Создаем датасет
    dataset_instance = sv.DetectionDataset(classes=classes_labels, 
                                 images=list_of_images,
                                 annotations={list_of_images[i]: None 
                                            for i 
                                            in range(len(list_of_images))})
    return dataset_instance

In [7]:
def perfom_detection_pipeline(model_name: Literal['yolov8l-worldv2.pt', 
                                                  'yolov8m-worldv2.pt', 
                                                  'yolov8s-worldv2.pt', 
                                                  'yolov8x-worldv2.pt'],
                                output_dir: str, 
                                batch_size: int, 
                                confidence: float, 
                                image_size: tuple[int], 
                                nms_threshold: float, 
                                list_of_images: list[str],
                                prompts_list: list[str],
                                dataset_instance: sv.DetectionDataset):
    """
        Функция для выполнения детекций и сохранения метаданных о них в специальный дата-класс.

        Args
        ----------
        model_name: (Literal['yolov8l-worldv2.pt', ...]): тип модели YOLO-World (light, medium, etc),

        output_dir (str): название директории, 
        куда сбрасываются размеченные изображения (для визуальной проверки результатов),

        batch_size (int): количество изображений, обрабатываемых в одной пачке,

        confidence (float): параметр порогово значения для отображения детекций на изображении,

        image_size (tuple[int]): размер изображения (лучшая практика: 640x640),

        nms_threshold (float): порог для отображения пересекающихся детекций (в зависимости от задачи),

        list_of_images (list[str]): список относительных путей к изображениям,

        prompts_list (list[str]): список промптов к модели,
        
        dataset_instance (sv.DetectionDataset): дата-класс для хранения метаданных по детекциям
        для каждого изображения.

        Returns
        -------
        dataset_instance (sv.DetectionDataset): заполненный метаданными детекций дата-класс.
    """
    # Инициализируем модель
    model = YOLOWorld(model_name)

    # Подаем промпты в модель
    model.set_classes(prompts_list)

    # Итерация по списку изображений: обрабатываем их батчами
    for i in range(0, len(list_of_images), batch_size):
        batch = list_of_images[i:batch_size + i + 1] 
        # TODO если классов несколько - для каждого из них должен быть разный порог уверенности
        results = model.predict(batch, conf=confidence, imgsz=image_size, iou = nms_threshold)

        # Итерация по каждой детекции в батче
        for index in range(len(results)):
            # Получение детекций
            detections = sv.Detections.from_ultralytics(results[index])

            # Инициализация экземпляров классов для боксов и подписей
            annotator = sv.BoxAnnotator()
            label_annotator = sv.LabelAnnotator()

            # вывод доп информации (уверенность детекции)
            labels = [
                f"{confidence:.2f}"
                for confidence
                in detections.confidence
            ]

            # Подготавливаем картинку для разметки
            image_for_labeling = cv2.imread(batch[index])
            # Непосредственно, рисуем боксы и подписи
            image_with_bounding_boxes = annotator.annotate(scene = image_for_labeling, 
                                                detections = detections)
            annotated_image = label_annotator.annotate(scene = image_with_bounding_boxes, 
                                                    detections = detections, 
                                                    labels=labels)
            
            # Сохраняем размеченную картинку в специально отведенную директорию
            cv2.imwrite(f"{output_dir}/{os.path.basename(batch[index])}", annotated_image)

            # Вносим в дата-класс инфу по детекциям для конкретной картинки
            dataset_instance.annotations[batch[index]] = detections
            
    return dataset_instance

In [8]:
def create_dataset_in_yolo_format(detections_data: sv.DetectionDataset, dataset_dir_name: str) -> None:
    """
        Функция для преобразования дата-класса с метаданными детекций в датасет YOLO.

        Args
        ----------
        detections_data (sv.DetectionDataset): инстанс дата-класса с метаданными детекций,

        dataset_dir_name (str): название папки с датасетом.
    """
    # Разбиваем датасет на train, test и val
    train_dataset, test_dataset = detections_data.split(split_ratio=0.8, random_state=42, shuffle=True)
    
    # Дополнительно разобьем test, чтобы получить val
    test_dataset, val_dataset = test_dataset.split(split_ratio=0.5, random_state=42)

    dct_of_datasets = {'train': train_dataset,
                   'test': test_dataset,
                   'val': val_dataset}
    
    for dataset_part, dataset_instance in dct_of_datasets.items():
        # Экспортируем в YOLO-формат (data_yaml генерируется невалидный для CVAT, нужно создавать самому)
        dataset_instance.as_yolo(
        images_directory_path=f"{dataset_dir_name}/{dataset_part}/images",
        annotations_directory_path=f"{dataset_dir_name}/{dataset_part}/labels",
        )

In [ ]:
def create_json(detection_data: sv.DetectionDataset, filename: str) -> None:
    """
        Функция для формирования JSON-файлика с данными детекций для каждого изображения
        (в формате xyxy-координат)

        Args
        ----------
        detections_data (sv.DetectionDataset): инстанс дата-класса с метаданными детекций,

        filename (str): желаемое название JSON-файла.
    """
    detecton_dict = {}
    coords_lst = ['x1', 'y1', 'x2', 'y2']
    list_of_keys = list(detection_data.annotations.keys())
    for key in list_of_keys:
        metadata_for_image = {}
        for index in range(len(coords_lst)):
            if len(detection_data.annotations[key]) == 0:
                continue
            metadata_for_image[coords_lst[index]] = float(detection_data.annotations[key].xyxy[0][index])
        detecton_dict[key] = metadata_for_image
    with open(filename, 'w', encoding = 'UTF-8') as my_file:
        json.dump(detecton_dict, my_file, indent=4, ensure_ascii=False)

In [9]:
check_output_dir_availability(output_dir)

detection_data = create_instance_of_detection_dataset(list_of_classes_labels, list_of_images)

filled_detection_data = perfom_detection_pipeline(model_name,
                                                output_dir, 
                                                batch_size, 
                                                confidence, 
                                                image_size, 
                                                nms_threshold, 
                                                list_of_images,
                                                list_of_prompts,
                                                detection_data)
create_dataset_in_yolo_format(filled_detection_data, yolo_dataset_name)
create_json(filled_detection_data, 'detections_data.json')


0: 640x640 1 
Russian car license plate: 
- white rectangular-shape metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A 123 BC 77 RUS
- mounted with visible bolts/screws
, 62.6ms
1: 640x640 1 
Russian car license plate: 
- white rectangular-shape metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A 123 BC 77 RUS
- mounted with visible bolts/screws
, 62.6ms
2: 640x640 2 
Russian car license plate: 
- white rectangular-shape metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A 123 BC 77 RUS
- mounted with visible bolts/screws
s, 62.6ms
3: 640x640 1 
Russian car license plate: 
- white rectangular-shape metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A 123 BC 77 RUS
- mounted with visible bolts/screws
, 62.6ms
4: 640x640 1 
Russian car license plate: 
- white rectangu